Activity 2.6 -- Working with COVID 19 and World Bank Data
=========================================================

In this activity, you will explore relationships between various World
Bank indicators for countries and their corresponding COVID death rates.
First you need to download data on COVID-19 (see links and instructions
below) and the selected indicators from the Open World Bank data
available at <https://data.worldbank.org>.

**COVID data set source:** <https://coviddata.github.io/coviddata/#csvs>

**Tasks.** Use pandas and dfply to perform each of the following.

1.  Download the raw **time\_series\_covid19\_confirmed\_global.csv**
    dataset.

2.  Inspect the data and discuss the need to reshape. 

In [1]:
import pandas as pd
from dfply import *
from more_dfply import *

covid = pd.read_csv("/home/dsteberg1/activity_2_5_COVID_and_World_Bank_Data/data/time_series_covid19_confirmed_global.txt")
covid.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/3/22,9/4/22,9/5/22,9/6/22,9/7/22,9/8/22,9/9/22,9/10/22,9/11/22,9/12/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,193912,194163,194355,194614,195012,195298,195471,195631,195925,196182
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,330062,330193,330221,330283,330516,330687,330842,330948,331036,331053
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,270426,270443,270461,270476,270489,270507,270522,270532,270539,270551
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,46027,46027,46027,46027,46113,46113,46113,46113,46113,46113
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,102636,102636,102636,102636,102636,102636,103131,103131,103131,103131


We definitely should stack all the dates into therir own column called date with the type datetype

3.  Write a single pipe that reshapes the data, sets the dtype of the date column, and extracts various date parts.
    1. To change the `dtype` of the date column, `date = X.date.astype('datetime64')`
    2. To extract the year and month, use the `X.date.dt.year` and `X.date.dt.month` attributes. This will need to happen in a separate `mutate` 

In [2]:
covid_reshape = (covid 
    >> gather("date", "cases", columns_from("1/22/20"))
    >> mutate(date = X.date.astype("datetime64"))
    >> mutate(year = X.date.dt.year,
              month = X.date.dt.month)
    >> drop(X.date)
)

covid_reshape.head()

,Province/State,Country/Region,Lat,Long,cases,year,month
0,NaN,Afghanistan,33.93911,67.709953,0,2020,1
1,NaN,Albania,41.15330,20.168300,0,2020,1
2,NaN,Algeria,28.03390,1.659600,0,2020,1
3,NaN,Andorra,42.50630,1.521800,0,2020,1
4,NaN,Angola,-11.20270,17.873900,0,2020,1


### World Bank Links Development Indicators

<https://databank.worldbank.org/source/world-development-indicators>

#### Constructing a data set.

First you need to construct a data set as follows

1.  Expand the Country tab and select all.

<img src="./img/media/image1.png" width="300">

2.  Click on the Series tab, search for *Health* and select the
    following indicators. **Feel free to add additional indicators!**

<img src="img/media/image2.png" width="300">

3.  Click on the Time tab and select 2020 and 2021.

4.  Click apply changes in the floating dialog.

<img src="img/media/image3.png" width="300">

5.  Select CSV from the Download Options button and save the data folder

<img src="img/media/image4.png" width="100">

#### Tasks

Use pandas and dfply to perform each of the following.

1.  Inspect the World Bank data and discuss the need to reshape. 

**Hints:** 

* You should apply `fix_names` from `more_dfply` to clean up the column names.
* This table needs to be reshaped twice




In [3]:
health = pd.read_csv("/home/dsteberg1/activity_2_5_COVID_and_World_Bank_Data/data/Data_Extract_From_World_Development_Indicators (1)/22dc457e-13ba-4569-a988-eaec8750d737_Data.csv")
health.head()

,Country Name,Country Code,Series Name,Series Code,2018 [YR2018],2019 [YR2019]
0,Afghanistan,AFG,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,14.12674332,13.24220181
1,Afghanistan,AFG,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,69.99860382,65.80603027
2,Afghanistan,AFG,Domestic general government health expenditure...,SH.XPD.GHED.GD.ZS,0.54922014,1.08443093
3,Afghanistan,AFG,Domestic general government health expenditure...,SH.XPD.GHED.PC.CD,2.72140895,5.38899046
4,Afghanistan,AFG,Domestic private health expenditure (% of curr...,SH.XPD.PVTD.CH.ZS,76.319664,79.39915466


We will need to stack the years and then spread out the series name column into seperate columns. 

2.  Write a single pipe that reshapes the data, sets the dtype of the date column, and extracts various date parts.
    1. You can use the `replace` method to clean up the year column.  See lecture 3.1 for details.

In [4]:
health_reshape = (health
    >> gather("year", "count", columns_from("2018 [YR2018]"), add_id = True)
    >> spread("Series Name", "count")
    >> select(columns_between("Country Name", "year"),
              columns_between("Current health expenditure (% of GDP)", "Domestic private health expenditure per capita (current US$)"))
)

health_reshape.head()

,Country Name,Country Code,Series Code,_ID,year,Current health expenditure (% of GDP),Current health expenditure per capita (current US$),Domestic general government health expenditure (% of GDP),Domestic general government health expenditure per capita (current US$),Domestic private health expenditure (% of current health expenditure),Domestic private health expenditure per capita (current US$)
0,Afghanistan,AFG,SH.XPD.CHEX.GD.ZS,0,2018 [YR2018],14.12674332,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,SH.XPD.CHEX.GD.ZS,0,2019 [YR2019],13.24220181,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,SH.XPD.CHEX.PC.CD,1,2018 [YR2018],NaN,69.99860382,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,SH.XPD.CHEX.PC.CD,1,2019 [YR2019],NaN,65.80603027,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,SH.XPD.GHED.GD.ZS,2,2018 [YR2018],NaN,NaN,0.54922014,NaN,NaN,NaN


### Investigate joining on country

Before we can proceed, we need to make sure that the columns used to join the data--namely the country--actually match.  Do this by

1. For each table, select just the country columns and make sure the column names match.
2. Perform a full outer join and filter on rows that didn't match (i.e. with a missing value in either column).
3. Determine any transformations needed to make the entries match.
4. Transform each of the original table as need (column names and/or problematic entries.

In [9]:
covid_country = (covid_reshape
    >> select(X["Country/Region"])
    >> rename(country = X["Country/Region"])
    >> mutate(file = "covid")
    >> distinct
)

covid_country.head()

,country,file
0,Afghanistan,covid
1,Albania,covid
2,Algeria,covid
3,Andorra,covid
4,Angola,covid


In [8]:
health_country = (health_reshape
    >> select(X["Country Name"])
    >> rename(country = X["Country Name"])
    >> mutate(file = "world_bank")
    >> distinct
)

health_country.head()

,country,file
0,Afghanistan,world_bank
12,Africa Eastern and Southern,world_bank
24,Africa Western and Central,world_bank
36,Albania,world_bank
48,Algeria,world_bank


In [10]:
joined_table = (covid_country
    >> outer_join(health_country, by = "country")
)

joined_table.head()

,country,file_x,file_y
0,Afghanistan,covid,world_bank
1,Albania,covid,world_bank
2,Algeria,covid,world_bank
3,Andorra,covid,world_bank
4,Angola,covid,world_bank


In [12]:
set_diff = (joined_table
    >> filter_by(X.country.notna())
    >> filter_by(X.file_x.isna() | X.file_y.isna())
    >> distinct
    >> arrange(X.country)
)

set_diff.head()

,country,file_x,file_y
199,Africa Eastern and Southern,NaN,world_bank
200,Africa Western and Central,NaN,world_bank
201,American Samoa,NaN,world_bank
5,Antarctica,covid,NaN
202,Arab World,NaN,world_bank


In [13]:
set_diff.to_csv("./data/set_diff.csv")

## Join and visualize 

Finally, you should use pandas and dfply to join these two data sets together, then create some interesting visualization using seaborn.

In [6]:
import seaborn as sns

### Deliverables
To complete this part of the activity, you need to submit the following.

1.  A link to this notebook including all discussion and code requests
    above.

2.  A csv file containing your final dataset. **Hint.** You can use the
    to\_csv method on the final data frame.

In [7]:
# Code for writing the data here